<img src="mioti.png" style="height: 100px">
<center style="color:#888">Módulo Data Science in IoT<br/>Asignatura: Data preprocessing</center>
# Challenge S2: Anonimización AirBnbMadrid

## Objetivos

En este challenge nos enfrentaremos a un dataset real, que contiene los datos de los alojamientos disponibles de Airbnb para la comunidad de Madrid. Nuestro objetivo en el challenge es anonimizarlo y convertir el dataset de AirBnBMadrid a AirBnBValladolid.

## Configuración del entorno

In [1]:
%matplotlib inline

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random


from faker import Faker
fake = Faker('es_ES')

## Carga de los datos
Esta vez va a ser fácil, vamos a importar los datos de un fichero csv, utilizaremos la función read_csv que nos proporciona la libreria de pandas.

In [2]:
df = pd.read_csv('dataset_airbnb_madrid.csv')

Como este dataset es muy complejo, vamos a quedarnos con un subconjunto de columnas para este challenge:

In [3]:
df = df[['id', 'listing_url', 'name', 'summary', 'price', 'weekly_price', 'zipcode', 'country', 'latitude', 'longitude']]

## Comprensión del dataset

Una vez cargados los datos debemos inspeccionarlos, y entender que datos contiene cada una de las columnas:

Describe por cada columna:

* ¿Qué contiene?
* ¿Cual es el rango de los datos?
* ¿Contiene datos sensibles?
* ¿Depende de otras?

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13335 entries, 0 to 13334
Data columns (total 10 columns):
id              13335 non-null int64
listing_url     13335 non-null object
name            13335 non-null object
summary         12846 non-null object
price           13335 non-null object
weekly_price    3512 non-null object
zipcode         12896 non-null object
country         13334 non-null object
latitude        13335 non-null float64
longitude       13335 non-null float64
dtypes: float64(2), int64(1), object(7)
memory usage: 1.0+ MB


In [5]:
df.head()

,id,listing_url,name,summary,price,weekly_price,zipcode,country,latitude,longitude
0,7830063,https://www.airbnb.com/rooms/7830063,Quiet room in Plaza Mayor,Room in magnificent property in the historic c...,$42.00,$300.00,28005,Spain,40.412275,-3.708718
1,9898596,https://www.airbnb.com/rooms/9898596,Homely apartment in the heart of Madrid,"Spacious apartment for up to 10 people, with a...",$135.00,NaN,28005,Spain,40.411093,-3.708985
2,15334645,https://www.airbnb.com/rooms/15334645,Piso Muy Luminoso en pleno centro de Madrid,"Lugares de interés: Casa Lucio, Cine Doré, Cal...",$81.00,NaN,28005,Spain,40.413587,-3.708945
3,1307795,https://www.airbnb.com/rooms/1307795,Rent room in the heart of Madrid,NaN,$43.00,$240.00,28013,Spain,40.419936,-3.709180
4,17410608,https://www.airbnb.com/rooms/17410608,Luxury duplex penthouse in historic building,Amazing duplex penthouse in a historic buildin...,$50.00,NaN,28005,Spain,40.410894,-3.712537


### id

In [6]:
df['id'].value_counts() 
# Tendremos un único ID para cada entrada, por lo que se aplicará una anonimización de cada una de las filas
#para la columna ID.
# Municipios cuyo nombre contiene una 'a'
#df_mask_municipio_y_nombre = (df['summary'].isin(['Madrid'])

5408767     1
16702606    1
13687099    1
14660403    1
13638132    1
729905      1
9568177     1
6413102     1
9880365     1
12120875    1
18082600    1
4675111     1
6597408     1
1757983     1
16040733    1
3527783     1
5169945     1
3811701     1
10556183    1
3771156     1
14799631    1
4381454     1
7110133     1
6299647     1
5440261     1
9323325     1
9603908     1
17439558    1
13814652    1
16046882    1
           ..
17397156    1
5227938     1
14800288    1
15994331    1
17966558    1
11638814    1
15592951    1
6661646     1
16856588    1
4132363     1
17956357    1
10843652    1
3853827     1
17548802    1
548353      1
7710206     1
11179512    1
1172982     1
14233055    1
8158709     1
6092276     1
14601715    1
644592      1
8986089     1
14685672    1
6862740     1
6815205     1
17587681    1
12965344    1
15892480    1
Name: id, Length: 13335, dtype: int64

### listing_url

In [7]:
df.duplicated(subset='listing_url', keep=False)
#Se observa que listing_url se identifica por ID. Con el comando de arriba se ha verificado que no hay duplicados;
#por lo tanto se puede considerar una estrategia de anonimización similar al ID, es decir una única url por entrada

0        False
1        False
2        False
3        False
4        False
5        False
6        False
7        False
8        False
9        False
10       False
11       False
12       False
13       False
14       False
15       False
16       False
17       False
18       False
19       False
20       False
21       False
22       False
23       False
24       False
25       False
26       False
27       False
28       False
29       False
         ...  
13305    False
13306    False
13307    False
13308    False
13309    False
13310    False
13311    False
13312    False
13313    False
13314    False
13315    False
13316    False
13317    False
13318    False
13319    False
13320    False
13321    False
13322    False
13323    False
13324    False
13325    False
13326    False
13327    False
13328    False
13329    False
13330    False
13331    False
13332    False
13333    False
13334    False
Length: 13335, dtype: bool

### name

In [8]:
df['name'].str.contains('Madrid').value_counts()
#Aunque el nombre de la dirección podría ser no representativo porque en principio las calles no son únicas por ciudad:
#se chequean las entradas que contienen la string 'Madrid'; de modo que a esas se les aplicará anonimización para
# que no sean identificables por la ciudad de Madrid.

False    10602
True      2733
Name: name, dtype: int64

### summary

In [9]:
df['summary'].str.contains('Madrid').value_counts()
# Desde el punto de vista summary, se observa que la mitad de los datos se podrian mapear con la variable Madrid dentro del string

False    6528
True     6318
Name: summary, dtype: int64

### price

In [10]:
df['price'].isnull()
#La columna precio siempre tiene un valor para cada uno de los pisos/habitaciones; el cual puede ser similar para
#varias entradas

0        False
1        False
2        False
3        False
4        False
5        False
6        False
7        False
8        False
9        False
10       False
11       False
12       False
13       False
14       False
15       False
16       False
17       False
18       False
19       False
20       False
21       False
22       False
23       False
24       False
25       False
26       False
27       False
28       False
29       False
         ...  
13305    False
13306    False
13307    False
13308    False
13309    False
13310    False
13311    False
13312    False
13313    False
13314    False
13315    False
13316    False
13317    False
13318    False
13319    False
13320    False
13321    False
13322    False
13323    False
13324    False
13325    False
13326    False
13327    False
13328    False
13329    False
13330    False
13331    False
13332    False
13333    False
13334    False
Name: price, Length: 13335, dtype: bool

In [11]:
df['price'].head()
#Se puede observar que el precio price puede variar en rangos dependiendo del piso o lugar; 

0     $42.00
1    $135.00
2     $81.00
3     $43.00
4     $50.00
Name: price, dtype: object

In [12]:
print(df['price'].max())
print(df['price'].min())
#Se observa que las funciones no aplican para el tipo string

$99.00
$1,000.00


### weekly_price

In [13]:
df['weekly_price'].isnull().value_counts()
#Se observa que hay varios casos para los que no aplica un weekly price.

True     9823
False    3512
Name: weekly_price, dtype: int64

### zipcode

In [14]:
df['zipcode']

0        28005
1        28005
2        28005
3        28013
4        28005
5        28013
6        28013
7        28005
8        28005
9        28013
10       28005
11       28013
12       28013
13       28013
14       28013
15       28008
16       28013
17       28005
18       28013
19       28005
20       28005
21       28013
22       28005
23       28005
24       28005
25       28005
26       28005
27       28005
28       28005
29       28013
         ...  
13305    28042
13306    28042
13307    28042
13308    28042
13309    28042
13310    28042
13311    28042
13312    28042
13313    28042
13314    28042
13315    28042
13316    28042
13317    28042
13318    28042
13319    28042
13320    28042
13321    28042
13322    28042
13323    28042
13324    28042
13325    28042
13326    28042
13327    28042
13328    28042
13329    28042
13330    28042
13331    28042
13332    28830
13333    28224
13334    28830
Name: zipcode, Length: 13335, dtype: object

### Country

In [15]:
df['country'].str.contains('Spain').value_counts()
#todas las eentradas contienen el país correcto para este ejercicio; excepto una que se identificará para 
#la anonimización correspondiente

True     13333
False        1
Name: country, dtype: int64

### latitude y longitude

## Dependencias entre variables
A continuación os propongo hacer una matriz de dependencias para analizar que variables dependen entre sí y analizar que grupos de variables existen.

| Depende de   | id | listing_url | name | summary | price | weekly_price | zipcode | country | lat y long |
|---           |--- |---          |---   |---      |---    |---           |---      |---      |---         | 
| id           | x  |     x       |      |         |       |              |         |         |            | 
| listing_url  |    |             |      |         |       |              |         |         |            | 
| name         |    |             |   x  |     x   |   x   |              |         |         |            | 
| summary      |    |             |      |     x   |   x   |              |         |         |            | 
| price        |    |             |      |         |       |        x     |         |         |            | 
| weekly_price |    |             |      |         |       |        x     |         |         |            | 
| zipcode      |    |             |      |         |   x   |              |         |    x    |      x      
| country      |    |             |      |         |       |              |    x    |         |      x     | 
| lat y long   |    |             |      |         |   x   |              |    x    |    x    |      x     | 



## Estrategia de anonimización

A partir de aquí por cada grupo de variables dependientes determina cual es la estrategia de anonimización más adecuada y aplícala.

In [16]:
df

,id,listing_url,name,summary,price,weekly_price,zipcode,country,latitude,longitude
0,7830063,https://www.airbnb.com/rooms/7830063,Quiet room in Plaza Mayor,Room in magnificent property in the historic c...,$42.00,$300.00,28005,Spain,40.412275,-3.708718
1,9898596,https://www.airbnb.com/rooms/9898596,Homely apartment in the heart of Madrid,"Spacious apartment for up to 10 people, with a...",$135.00,NaN,28005,Spain,40.411093,-3.708985
2,15334645,https://www.airbnb.com/rooms/15334645,Piso Muy Luminoso en pleno centro de Madrid,"Lugares de interés: Casa Lucio, Cine Doré, Cal...",$81.00,NaN,28005,Spain,40.413587,-3.708945
3,1307795,https://www.airbnb.com/rooms/1307795,Rent room in the heart of Madrid,NaN,$43.00,$240.00,28013,Spain,40.419936,-3.709180
4,17410608,https://www.airbnb.com/rooms/17410608,Luxury duplex penthouse in historic building,Amazing duplex penthouse in a historic buildin...,$50.00,NaN,28005,Spain,40.410894,-3.712537
5,9141653,https://www.airbnb.com/rooms/9141653,Center/Gran via apartment exclusive,"Apartament exceptionally located, next to the ...",$144.00,NaN,28013,Spain,40.419378,-3.708383
6,1320674,https://www.airbnb.com/rooms/1320674,City center! Opera-Sol-Plaza Mayor,"Illuminated, very functional, great location (...",$60.00,NaN,28013,Spain,40.416376,-3.710314
7,15118056,https://www.airbnb.com/rooms/15118056,COCOBOLO CENTER HOUSE WITH FREE PARKING,"Un hogar para disfrutar, rodeado de zonas verd...",$68.00,NaN,28005,Spain,40.414209,-3.716072
8,14870504,https://www.airbnb.com/rooms/14870504,Habitacion 1 en el Madrid de los Austrias,Lugares de interés: el apartamento esta situad...,$29.00,NaN,28005,Spain,40.415185,-3.713099
9,17049729,https://www.airbnb.com/rooms/17049729,APARTAMENTO GRAN VIA,BONITO APARTAESTUDIO UBICADO EN EL CORAZON DE ...,$60.00,NaN,28013,Spain,40.421408,-3.707650


In [17]:
## Primero, id y listing url se anonimizaran de forma conjunta al ser dependientes:
def anon_id(row_id):
    #Primero asignamos un id aleatorio con el metodo numerify de fake para cada id
    #En el siguiente paso, se añade ese id como parte de la concatenacion de la URL
    row_id['id'] = fake.numerify()      
    id = str(row_id['id'])
    row_id['listing_url'] = 'https://www.airbnb.com/rooms/' + id
    return row_id

df = df.apply(anon_id, axis=1)
df

,id,listing_url,name,summary,price,weekly_price,zipcode,country,latitude,longitude
0,050,https://www.airbnb.com/rooms/050,Quiet room in Plaza Mayor,Room in magnificent property in the historic c...,$42.00,$300.00,28005,Spain,40.412275,-3.708718
1,512,https://www.airbnb.com/rooms/512,Homely apartment in the heart of Madrid,"Spacious apartment for up to 10 people, with a...",$135.00,NaN,28005,Spain,40.411093,-3.708985
2,155,https://www.airbnb.com/rooms/155,Piso Muy Luminoso en pleno centro de Madrid,"Lugares de interés: Casa Lucio, Cine Doré, Cal...",$81.00,NaN,28005,Spain,40.413587,-3.708945
3,215,https://www.airbnb.com/rooms/215,Rent room in the heart of Madrid,NaN,$43.00,$240.00,28013,Spain,40.419936,-3.709180
4,809,https://www.airbnb.com/rooms/809,Luxury duplex penthouse in historic building,Amazing duplex penthouse in a historic buildin...,$50.00,NaN,28005,Spain,40.410894,-3.712537
5,043,https://www.airbnb.com/rooms/043,Center/Gran via apartment exclusive,"Apartament exceptionally located, next to the ...",$144.00,NaN,28013,Spain,40.419378,-3.708383
6,185,https://www.airbnb.com/rooms/185,City center! Opera-Sol-Plaza Mayor,"Illuminated, very functional, great location (...",$60.00,NaN,28013,Spain,40.416376,-3.710314
7,736,https://www.airbnb.com/rooms/736,COCOBOLO CENTER HOUSE WITH FREE PARKING,"Un hogar para disfrutar, rodeado de zonas verd...",$68.00,NaN,28005,Spain,40.414209,-3.716072
8,741,https://www.airbnb.com/rooms/741,Habitacion 1 en el Madrid de los Austrias,Lugares de interés: el apartamento esta situad...,$29.00,NaN,28005,Spain,40.415185,-3.713099
9,232,https://www.airbnb.com/rooms/232,APARTAMENTO GRAN VIA,BONITO APARTAESTUDIO UBICADO EN EL CORAZON DE ...,$60.00,NaN,28013,Spain,40.421408,-3.707650


In [18]:
## El segundo paso, sera cambiar la palabra Madrid en name y summary por Valladolid, 
##teniendo en cuenta los casos nulos para el summary en la excepci
def anon_name(row_name):
    row_name['name'] = row_name['name'].replace("Madrid", "Valladolid")  
    try:
        row_name['summary'] = row_name['summary'].replace('Madrid', 'Valladolid')
    except:
        row_name['summary'] = 'nice place within center of Valladolid'
    return row_name

df = df.apply(anon_name, axis=1)

In [19]:
#Para anonimizar el precio se divide entre 3 como referencia entre el precio de Valladolid y Madrid
#Además se asume el weekly price para que sea 7 veces el precio por noche
def anon_price(row_price):
    row_price['price'] = row_price['price'].replace(",", "")
    row_price['price'] = row_price['price'].replace("$", "")
    row_price['price'] = float(row_price['price'])
    row_price['price'] = round(row_price['price']/3, 3)
    #weekly Price
    row_price['weekly_price'] = row_price['price']*7
    
    return row_price

df = df.apply(anon_price, axis=1)

In [20]:
#Se asigna el prefijo de Valladolid y los siguientes numeros de forma aleatoria; 
# y se asigna Spain para los posibles valores None
def anon_zipcode(row_zc):
    row_zc['country']=='Spain'
    row_zc['zipcode']= fake.numerify('47###')
    return row_zc

df = df.apply(anon_zipcode, axis=1)

In [21]:

def anon_coord(row_cd):
    row_cd['longitude'] = fake.geo_coordinate(center = '41.6522966', radius= 0.05)
    row_cd['latitude'] = fake.geo_coordinate(center = '-4.7285413', radius=0.05)
    return row_cd
    
df = df.apply(anon_coord, axis=1)
df

,id,listing_url,name,summary,price,weekly_price,zipcode,country,latitude,longitude
0,050,https://www.airbnb.com/rooms/050,Quiet room in Plaza Mayor,Room in magnificent property in the historic c...,14.000,98.000,47689,Spain,-4.689991,41.624678
1,512,https://www.airbnb.com/rooms/512,Homely apartment in the heart of Valladolid,"Spacious apartment for up to 10 people, with a...",45.000,315.000,47002,Spain,-4.706364,41.647756
2,155,https://www.airbnb.com/rooms/155,Piso Muy Luminoso en pleno centro de Valladolid,"Lugares de interés: Casa Lucio, Cine Doré, Cal...",27.000,189.000,47534,Spain,-4.721147,41.647068
3,215,https://www.airbnb.com/rooms/215,Rent room in the heart of Valladolid,nice place within center of Valladolid,14.333,100.331,47000,Spain,-4.750136,41.658535
4,809,https://www.airbnb.com/rooms/809,Luxury duplex penthouse in historic building,Amazing duplex penthouse in a historic buildin...,16.667,116.669,47504,Spain,-4.764228,41.662587
5,043,https://www.airbnb.com/rooms/043,Center/Gran via apartment exclusive,"Apartament exceptionally located, next to the ...",48.000,336.000,47927,Spain,-4.759345,41.697869
6,185,https://www.airbnb.com/rooms/185,City center! Opera-Sol-Plaza Mayor,"Illuminated, very functional, great location (...",20.000,140.000,47683,Spain,-4.750222,41.616863
7,736,https://www.airbnb.com/rooms/736,COCOBOLO CENTER HOUSE WITH FREE PARKING,"Un hogar para disfrutar, rodeado de zonas verd...",22.667,158.669,47664,Spain,-4.711188,41.613560
8,741,https://www.airbnb.com/rooms/741,Habitacion 1 en el Valladolid de los Austrias,Lugares de interés: el apartamento esta situad...,9.667,67.669,47697,Spain,-4.704521,41.683957
9,232,https://www.airbnb.com/rooms/232,APARTAMENTO GRAN VIA,BONITO APARTAESTUDIO UBICADO EN EL CORAZON DE ...,20.000,140.000,47056,Spain,-4.737614,41.613040


## Conclusiones

* ¿Qué ventajas / inconvenientes le ves a esta manera de anonimizar?
* ¿Cómo podríamos mejorar los algoritmos?
* ¿Cómo podríamos evaluar que una anonimización es buena?

In [22]:
## Conclusiones


##¿Qué ventajas / inconvenientes le ves a esta manera de anonimizar?

#Las ventajas de esta manera de anonimizar es que cada una de las variables tiene una asignación aleatoria,
#haciendo difícil encontrar el origen de los datos

##La principal desventaja de no tener en cuenta las posibles dependencias a la hora de realziar anonimización
##aleatoria es que se pierde las posibles relaciones de los datos originales: 
##Precio semanal/precio original, Precio/Coordenadas, Nombre/Precio Semanal...


#¿Cómo podríamos mejorar los algoritmos?
#Los algortimos de arrriba se podrían mejorar aplicando las reglas originales iniciales de las dependencias entre las variables.
#Después de verificar las dependencias entre las variables, se tendría que estudiar la correlacion y la
#dependencia estadística entre las mismas y, finalmente, aplicarlas en los algoritmos. 
#Ejemplos: mantner Relacion del week price vs price de los datos; aplicando reglas geograficas...


##¿Cómo podríamos evaluar que una anonimización es buena?
#Entiendo que el primer objetivo es mantener la encriptación de los datos originales, aunque también al mismo tiempo
#se debe mantener la relación y dependencias correspondientes de los datos de inicio